In [6]:
import pandas as pd
import numpy as np
import itertools
import functools
from nl_03_filter_model_score import filter_split_model_score
import importlib

### This script runs the neutral loss scripts for MALDI IMS data

1. nl_00 Calculates Mordred descriptors and FP4 fingerprints as bits.
2. nl_01 Filter HMDB_db for observed and parses METASPACE output.
3. nl_02 Joins HMDB_db and METASPACE output.
4. nl_03 Filters and searchs with direct and machine learning models.

### To do:
5. nl_04 deep learning
6. Thunder SVM, XGBoost
7. Calculate %NL observed by ID...
7.

### Observations:
1. 2912/3333 datasets a) did not error and b) had at least one id.  
2. 644,094 values identifications, the dataset with the most identifications had 5,820 and mean was 221 (median 131).
3. The data did not pass the Shapiro-Wilk test for normality, and nonparametrical statistics should be used.

In [105]:
# Merge pickles for each dataset.
master_df = pd.DataFrame()
counter = 0

for root, dirs, files in os.walk("/Users/dis/PycharmProjects/neutal_loss_2/public_water_pickle"):
    for file in files:
        if file.endswith(".pickle"):
            if counter % 500 == 0:
                print(counter)
            counter +=1
            current_df = pd.read_pickle(root + '/' + file)
            master_df = pd.concat([master_df, current_df], sort=True)

# If water
terse_headers_0 = ['formula', 'adduct', 'ds_id', 'has_no_loss', 'has_H2O',
                   'msm', 'fdr', 'off_sample', 'hmdb_ids', 'intensity_avg',
                   'msm_H2O', 'fdr_H2O', 'off_sample_H2O', 'intensity_avg_H2O',
                   'colocalization_H2O', 'loss_intensity_ratio_H2O', 'loss_intensity_share_H2O']

master_df[terse_headers_0].to_pickle('all_public_data.pickle')

0
500
1000
1500
2000
2500


In [4]:
# Loads merged pickles from previous step
dataset = 'all_public_data.pickle'  # A GB

In [ ]:
# To be built, downloads HMDB and calculates molecules

In [98]:
# nl_00: Generates FP4 descriptors ('bits') and Mordred descriptors
# 10420s
%run nl_00_hmdb_pre.py

  0%|          | 47/114100 [00:01<2:52:10, 11.04it/s]

/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  0%|          | 126/114100 [00:03<1:12:07, 26.34it/s]

/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  0%|          | 439/114100 [00:12<1:24:50, 22.33it/s]

...

/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|▋         | 7686/114100 [15:55<6:53:41,  4.29it/s]  

/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in doub

100%|██████████| 114100/114100 [4:54:22<00:00,  6.46it/s]   


Elapsed time:

18533.797726869583

Executed without error

all_public.pickle


In [106]:
# Loads pickles from previous step
input_df = pd.read_pickle('all_public_data.pickle')
hmdb_df = pd.read_pickle('all_public_hmdb.pickle') # 0.1 Gb
mord_norm_df = pd.read_pickle('mord_norm_hmdb_df.pickle')  # 0.8 Gb
bits_df = pd.read_pickle('bits_hmdb_df.pickle')  #0.2 Gb

In [109]:
# nl_01: Parses METASPACE output and filters HMDB for observed
%run nl_01_preprocess.py -m all_public_data.pickle -p all_public_hmdb.pickle -is_H2O True

Elapsed time (seconds):

22

Executed without error

all_public_data_output_01.pickle
all_public_data_hmdb_01.pickle


In [110]:
# Loads pickles from previous step
output_1 = pd.read_pickle('all_public_output_01.pickle')
hmdb_1 = pd.read_pickle('all_public_hmdb_01.pickle')

In [116]:
# nl_02: Joins METASPACE output and observed HMDB
%run nl_02_join.py --o all_public_output_01.pickle --h all_public_hmdb_01.pickle

Elapsed time:


Executed without error

2952.2901570796967
all_public_output_02.pickle


In [7]:
# Load X's if running machine learning
join_df_path = 'all_public_output_02.pickle' 
join_df = pd.read_pickle(join_df_path)  # 0.5 Gb
unique_hmdbs = list(join_df.hmdb_ids.unique())

mord_df = pd.read_pickle('mord_norm_hmdb_df.pickle') # 0.2 Gb
mord_df = mord_df[mord_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

bits_df = pd.read_pickle('bits_hmdb_df.pickle') # 0.8 Gb
bits_df = bits_df[bits_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

In [12]:
# nl_03: Filtering and machine learning models
target = ['H2O'] # ['H2O']
polarity = [1] # {1:positive, -1:negative] 
fdrs = [0.10] # [0.20, 0.10, 0.05]
colocalizations = [0.50] # [0.00, 0.50, 0.75]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O']
y = [global_ys[1]]
w = ['weight']

# Class of model:
models = ['direct', 'ml', 'deepchem']
model = [models[1]]

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

filter_model_df = pd.DataFrame(columns=['target', 'polarity', 'fdr', 'coloc',
                                       'model', 'submodel', 'params', 'X', 'y', 'w'])

# Specific inputs (X) for each model:
if model[0] is 'direct':
    submodel = ['']
    params = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X = [Xs[3]]
                              
    
elif model[0] is 'ml':
    submodels = ['random_forest', 'XGBoost', 'ThunderSVM']
    submodel = [submodels[2]]
    params = ['']
    Xs = ['bits', 'mord_norm'] 
    X = [Xs[0]]

elif model[0] is 'deepchem':
    submodels = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('MPNNModel','Weave') ] 
    submodel = [submodels[0]]    
    params = ['']
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    
    
model_list = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                      model, submodel, params, X, y, w]))
filter_param_df = pd.DataFrame(model_list, columns=['target', 'polarity', 'fdr', 'coloc',
                                                    'model', 'submodel', 'params', 
                                                    'X', 'y', 'w'])

filter_param_df

,target,polarity,fdr,coloc,model,submodel,params,X,y,w
0,H2O,1,0.1,0.5,ml,ThunderSVM,,bits,n_loss_wparent_H2O,weight


In [ ]:
model_results = filter_split_model_score(filter_param_df, join_df_path, mord_df, bits_df,
                                         single_fold_group)

start 0
splitting 0
modeling 0
ml
mL_dfs
train_y: (699148,)
train_w: (699148,)
train_X: (699148, 1024)


In [10]:
model_results.round(3)

,target,polarity,fdr,coloc,model,submodel,params,X,y,w,...,spec,f1,test_n,test_true,tp,tn,fp,fn,acc_train,acc_test
0,H2O,1,0.1,0.5,ml,XGBoost,,bits,n_loss_wparent_H2O,weight,...,0.998,0.04,699148,2789,1792,611212,997,85147,0.877,0.877


In [11]:
all_model_results = pd.read_pickle('all_model_results.pickle')
all_model_results = pd.concat([all_model_results, model_results])
all_model_results.to_pickle('all_model_results.pickle')
all_model_results.round(3)

,target,polarity,fdr,coloc,model,submodel,params,X,y,w,...,spec,f1,test_n,test_true,tp,tn,fp,fn,acc_train,acc_test
0,H2O,1,0.10,0.50,direct,,,trues,n_loss_wparent_H2O,weight,...,0.000,0.165,349909,349909,31561,0,318348,0,0.125,0.090
1,H2O,1,0.10,0.50,direct,,,falses,n_loss_wparent_H2O,weight,...,1.000,0.000,349909,0,0,318348,0,31561,0.875,0.910
2,H2O,1,0.10,0.50,direct,,,rando,n_loss_wparent_H2O,weight,...,0.515,0.153,349909,169933,15417,163832,154516,16144,0.503,0.512
3,H2O,1,0.10,0.50,direct,,,H2O_Present,n_loss_wparent_H2O,weight,...,0.738,0.274,349909,101594,18233,234987,83361,13328,0.685,0.724
4,H2O,1,0.10,0.50,direct,,,n_loss_wparent_H2O,n_loss_wparent_H2O,weight,...,1.000,1.000,349909,31561,31561,318348,0,0,1.000,1.000
5,H2O,-1,0.10,0.50,direct,,,trues,n_loss_wparent_H2O,weight,...,0.000,0.265,93330,93330,14246,0,79084,0,0.130,0.153
6,H2O,-1,0.10,0.50,direct,,,falses,n_loss_wparent_H2O,weight,...,1.000,0.000,93330,0,0,79084,0,14246,0.870,0.847
7,H2O,-1,0.10,0.50,direct,,,rando,n_loss_wparent_H2O,weight,...,0.493,0.235,93330,47320,7233,38997,40087,7013,0.506,0.495
8,H2O,-1,0.10,0.50,direct,,,H2O_Present,n_loss_wparent_H2O,weight,...,0.596,0.199,93330,37036,5092,47140,31944,9154,0.510,0.560
9,H2O,-1,0.10,0.50,direct,,,n_loss_wparent_H2O,n_loss_wparent_H2O,weight,...,1.000,1.000,93330,14246,14246,79084,0,0,1.000,1.000


To debug:
1. ml functions: SVM
2. deepchem functions: Weave, MPNN, transfer to nl03.  predictprob
3. Pass arguments named_tuples to functions for opt?
4. y = Scrambled as option for same balance of classes instead of rando

In [ ]:
# Check for nans in mord_df